# Extract the text from the JSON from Stemmaweb

In [2]:
import json

In [3]:
file = "notre_besoin.json"

with open(file, "r") as f:
    data = json.load(f)

In [26]:
full_text = {}
for dt in data["alignment"]:
    text = ""
    for token in dt["tokens"]:
        if token:
            text += token["text"] + " "
    full_text[dt["witness"]] = text

In [10]:
def get_texts(json_collation_path):
    """Retrieve the full texts from a JSON collation."""
    with open(json_collation_path, "r") as f:
        json_collation = json.load(f)
    full_texts = {}
    for alignment in json_collation["alignment"]:
        text = ""
        for token in alignment["tokens"]:
            if token:
                text += token["text"] + " "
        full_texts[alignment["witness"]] = text
    return full_texts

In [11]:
get_texts("notre_besoin.json")

{'A': "Notre besoin de consolation est impossible à rassasier de Stig Dagerman Je suis dépourvu de foi et ne puis donc être heureux , car un homme qui risque de craindre que sa vie ne soit errance absurde vers une mort certaine ne peut être heureux . Je n'ai reçu en héritage ni dieu , ni point fixe sur la terre d'où je puisse attirer l'attention d'un dieu ... On ne m'a pas non plus légué la fureur bien déguisée du sceptique , les ruses des Sioux du rationaliste ou la candeur ardente de l'athée . Je n'ose donc jeter la pierre ni à cle qui croit des choses qui ne m'inspirent que le doute , ni à celui qui cultive son doute comme si celui-ci n'était pas , lui aussi , entouré de ténèbres . Cette pierre m'atteindrait moi-même car je suis bien certain d'une chose : le besoin de consolation que connait l'être humain est impossible à rassasier . En ce qui me concerne , je traque la consolation comme le chasseur traque le gibier . Partout où je crois l'apercevoir dans la forêt , je tire . Souven

In [47]:
from pathlib import Path

files = {}
for text in Path("notrebesoin/").glob("*"):
    print(text)
    with open(text, "r", encoding='latin-1') as f:
        files[text.stem] = f.read().replace("\n", " ")


notrebesoin/n12.txt
notrebesoin/n13.txt
notrebesoin/n11.txt
notrebesoin/n10.txt
notrebesoin/n8.txt
notrebesoin/n9.txt
notrebesoin/n7.txt
notrebesoin/n6.txt
notrebesoin/n4.txt
notrebesoin/n5.txt
notrebesoin/n1.txt
notrebesoin/n2.txt
notrebesoin/n3.txt


In [61]:
for text in full_text:
    print(text)
    print(full_text[text])
    print(files[text])


{'A': "Notre besoin de consolation est impossible à rassasier de Stig Dagerman Je suis dépourvu de foi et ne puis donc être heureux , car un homme qui risque de craindre que sa vie ne soit errance absurde vers une mort certaine ne peut être heureux . Je n'ai reçu en héritage ni dieu , ni point fixe sur la terre d'où je puisse attirer l'attention d'un dieu ... On ne m'a pas non plus légué la fureur bien déguisée du sceptique , les ruses des Sioux du rationaliste ou la candeur ardente de l'athée . Je n'ose donc jeter la pierre ni à cle qui croit des choses qui ne m'inspirent que le doute , ni à celui qui cultive son doute comme si celui-ci n'était pas , lui aussi , entouré de ténèbres . Cette pierre m'atteindrait moi-même car je suis bien certain d'une chose : le besoin de consolation que connait l'être humain est impossible à rassasier . En ce qui me concerne , je traque la consolation comme le chasseur traque le gibier . Partout où je crois l'apercevoir dans la forêt , je tire . Souven

In [100]:
import textdistance


text_distance = {}
closest_text = {}
for text1 in full_text:
    best_min = 0
    text_distance[text1] = {}
    for text2 in files:
        text_distance[text1][text2] = textdistance.jaccard(remove_punctuation((full_text[text1]).lower()), 
                                                            remove_punctuation(files[text2]).lower())
        if text_distance[text1][text2] > best_min:
            best_min = text_distance[text1][text2]
            best = text2
    closest_text[best] = text1

In [84]:
def different_words(s1, s2):
    # Split the strings into sets of words
    set1 = set(s1.split())
    set2 = set(s2.split())
    
    # Compute the difference between the sets
    diff1 = set1 - set2
    diff2 = set2 - set1
    
    # Combine the differences
    different_words = diff1.union(diff2)
    
    return different_words


import string

def remove_punctuation(text):
    # Create a translation table that maps each punctuation character to None
    translator = str.maketrans('', '', string.punctuation)
    # Use the translate method to remove punctuation
    return text.translate(translator)


In [103]:
# Rename the content in the .nex file
with open("notrebesoin_groundtruth/notrebesoin.nex.txt", "r") as f:
    content = f.read()

In [108]:
for text in closest_text:
    content = content.replace(text, closest_text[text])

with open("notrebesoin_groundtruth/notrebesoin_gt.nex", "w") as f:
    f.write(content)

In [109]:
print(content)

begin data;
	dimensions ntax = 13 nchar = 1035;
	format symbols = "ABCDEF" labels = left;
	matrix
		U	AAAAAAAA?AA?AAAAAAAAAAAAAAAAAAAAAAA?AAAAAAAAAABAAAAABAAAAAAAAAAAADAAAAAAAAAAAAAAABAABAAAAAAAAAAAAAABAC?AAAABAAAAAAAAAABAAAAABAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAACAAAAAAAAAAAAAAAAAABAAAAAAABAAAABAAAAAAAAAAAAAAAAABAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAABAAAAABAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAABAAAAAAAB??AAABBBAAABBAAAAABAAAAABBAACAAABBABAAAAAABBCAAAAAAAABAAAAAAAAAAABAAAAAAAAAAAAAAAAAAAAAAAA?AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAABAAAAAB?????????AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAABAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAABABAAAAAAAAAAAAAAAAAAAAAAAAAAA????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????????

In [8]:
import json

def json_to_nexus(json_file_path, nexus_file_path):
    # Load JSON data
    with open(json_file_path, 'r') as f:
        data = json.load(f)
    
    # If the data is nested under a key like 'alignment', adjust accordingly
    if 'alignment' in data:
        alignment = data['alignment']
    else:
        alignment = data
    print("===")
    print(alignment)
    # Extract taxa and alignment data
    taxa = list(alignment.keys())
    num_chars = 0
    for taxon in taxa:
        if isinstance(alignment[taxon], list):
            if len(alignment[taxon]) > num_chars:
                num_chars = len(alignment[taxon])
        else:
            print(f"Error: Alignment for taxon '{taxon}' is not a list.")
            return
    
    # Build variant symbols for each locus
    variant_symbols = []
    for position in range(num_chars):
        # Collect words at this position from all taxa
        words = set()
        for taxon in taxa:
            try:
                word = alignment[taxon][position]
                if word != '':
                    words.add(word)
            except (IndexError, KeyError):
                pass
        # Order the words alphabetically
        words = sorted(words)
        # Assign symbols to words (A, B, C, ...)
        symbols = {}
        for idx, word in enumerate(words):
            symbols[word] = chr(ord('A') + idx)
        variant_symbols.append(symbols)
    
    # Build the Nexus data matrix
    nexus_matrix = {}
    for taxon in taxa:
        sequence = ''
        for position in range(num_chars):
            try:
                word = alignment[taxon][position]
                if word != '':
                    symbol = variant_symbols[position].get(word, '?')
                    sequence += symbol
                else:
                    sequence += '?'
            except IndexError:
                sequence += '?'
        nexus_matrix[taxon] = sequence
    
    # Write the Nexus file
    with open(nexus_file_path, 'w') as f:
        f.write('#NEXUS\n\n')
        f.write('BEGIN DATA;\n')
        f.write(f'    DIMENSIONS NTAX={len(taxa)} NCHAR={num_chars};\n')
        f.write('    FORMAT DATATYPE=STANDARD MISSING=? GAP=-;\n')
        f.write('    MATRIX\n')
        for taxon in taxa:
            f.write(f'    {taxon} {nexus_matrix[taxon]}\n')
        f.write('    ;\n')
        f.write('END;\n')

# Example usage:
json_to_nexus('notre_besoin.json', 'output.nex')

===
[{'witness': 'A', 'tokens': [{'text': 'Notre besoin de consolation est impossible à', 'language': 'French', 'rank': 1, 'normal_form': 'Notre besoin de consolation est impossible à', 'id': '182346', 'is_common': True, 'lexemes': '[{"is_disambiguated":"1","string":"Notre","language":"French","wordform_matchlist":["French // notre // cat@det type@poss pers@3 num@sing def@def"],"form":"French // notre // cat@det type@poss pers@3 num@sing def@def"},{"form":"French // besoin // cat@noun type@common num@sing","wordform_matchlist":["French // besoin // cat@noun type@common num@sing"],"language":"French","is_disambiguated":"1","string":"besoin"},{"form":"French // de // cat@prep","wordform_matchlist":["French // de // cat@prep"],"is_disambiguated":"1","string":"de","language":"French"},{"form":"French // consolation // cat@noun type@common num@sing","wordform_matchlist":["French // consolation // cat@noun type@common num@sing"],"string":"consolation","is_disambiguated":"1","language":"Frenc

AttributeError: 'list' object has no attribute 'keys'